In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 198 kB 69.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4234fba72a513f1ab2ed1fb5d4f4f53b48543bf2979a1e5cd97b304cee8d2697
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)

In [3]:
import math
from pyspark.sql.functions import *

In [19]:
#creamos una pequeña data
import random
data = [random.randint(1, 20) for _ in range(20)]
data

[20, 16, 13, 13, 5, 17, 12, 7, 20, 8, 12, 9, 17, 12, 16, 20, 15, 14, 19, 17]

In [6]:
#Estandarizacion
#Primero hallamnos la media, tambien la desviacion estandar, luego se realizamos la estandarizacion a todos los elementos
def standardization (data):
    Mean = data.mean()
    Standard_Deviation = data.stdev()
    ans = data.map(lambda x: (x-Mean)/(Standard_Deviation))
    return ans

In [20]:

dataRDD = sc.parallelize(data,3)
test = standardization(dataRDD)
print(test.collect())

[1.3647319560747138, 0.43948995195626384, -0.25444155113257366, -0.25444155113257366, -2.1049255593694736, 0.6708004529858763, -0.48575205216218614, -1.6423045573102486, 1.3647319560747138, -1.4109940562806362, -0.48575205216218614, -1.1796835552510236, 0.6708004529858763, -0.48575205216218614, 0.43948995195626384, 1.3647319560747138, 0.20817945092665133, -0.023131050102961168, 1.1334214550451014, 0.6708004529858763]


In [15]:
#Escalonamiento
#primeo hallamos el termino minimno y termino maxino, luego hallamos el escalonamiento
def staging (data):
    max = data.max()
    min = data.min()
    ans = data.map(lambda x: (x-min)/(max-min))
    return ans

In [23]:
dataRDD1 = sc.parallelize(data,3)
test1 = staging(dataRDD1)
print(test1.collect())

[1.0, 0.7333333333333333, 0.5333333333333333, 0.5333333333333333, 0.0, 0.8, 0.4666666666666667, 0.13333333333333333, 1.0, 0.2, 0.4666666666666667, 0.26666666666666666, 0.8, 0.4666666666666667, 0.7333333333333333, 1.0, 0.6666666666666666, 0.6, 0.9333333333333333, 0.8]


In [27]:
#primero hallamos la lista de cuadros , los sumamos y finamente escalonamos
def normalization(x):
    squared = x.map(lambda x:x * x)
    total_sum = squared.sum()
    val = math.sqrt(total_sum)
    ans = x.map(lambda xi :(xi/val))
    return ans 

In [28]:
dataRDD2 = sc.parallelize(data,3)
test2 = normalization(dataRDD2)
print(test2.collect())

[0.3032392174315614, 0.2425913739452491, 0.1971054913305149, 0.1971054913305149, 0.07580980435789035, 0.25775333481682716, 0.18194353045893683, 0.10613372610104649, 0.3032392174315614, 0.12129568697262455, 0.18194353045893683, 0.1364576478442026, 0.25775333481682716, 0.18194353045893683, 0.2425913739452491, 0.3032392174315614, 0.22742941307367104, 0.21226745220209298, 0.2880772565599833, 0.25775333481682716]


In [36]:
#producto de una tupla
def ProductI(X):
  return X[0]*X[1]

# agrupamos las dos listas en tublas, hallamos la suma del producto de las tuplas, hallamos la suma del cuadrado de amabos elementos de la tupla y hallamos la sililitud de cosenos 
def Similaritycos(v1,v2):
  v=list(zip( v1, v2))
  A1 = sc.parallelize(v1,3)
  A2 = sc.parallelize(v2,3)
  A3 = sc.parallelize(v,3)
  SumProd = A3.map(lambda x: ProductI(x)).sum()
  SumProdSqu = A1.map(lambda x:x**2).sum()
  SumProdSqu1 = A2.map(lambda x:x**2).sum()
  ans = (SumProd) / ( SumProdSqu**(1/2) * SumProdSqu1**(1/2) )
  return ans

In [40]:
v1=list(range(5,10))
v2=list(range(10,15))
print(Similaritycos(v1,v2))

0.9966368371814116
